# BenzDB tools

## Source code

### Module imports

In [1]:
import ipywidgets as w
import requests
import json
import base64 
import io
from PIL import Image
import IPython as ip
import jupyter_jsmol as Jsmol
import pandas as pd
import matplotlib.pyplot as plt

### Definition of criterion classes

#### Definition of the main criterion class

In [2]:
class Criterion:
    """ This class allows for representing criterions """
    
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        self.__key = key
        self.__description  = description     
    
    
    def display (self):
        """ displays the widget corresponding to the criterion """
        pass
    
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        pass
    
    
    def get_description (self) -> str:
        """ returns the description """
        return self.__description
    
    
    def get_key (self) -> str:
        """ returns the key related to the criterion """
        return self.__key
    
    
    def set_widget (self, w) -> None:
        """ sets the widget """
        self.__widget = w
    
    
    def get_widget (self):
        return self.__widget

#### Definition of criterion child classes

In [3]:
class Int_Criterion (Criterion):
    """ This class allows for representing criterions based on int value """
 
    def __init__ (self, key: str, description: str, min_value: int, max_value: int):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.BoundedIntText(value=min_value, min=min_value, max=max_value, layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>","<=","<",">",">="], value='not set', rows=1, layout={"width": "auto"})
                
        self.set_widget(w.HBox([self.__condition, self.__element]))
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [4]:
class Boolean_Criterion (Criterion):
    """ This class allows for representing criterions based on Boolean value """
 
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.BoundedIntText(value=1, min=0, max=1, layout={"width": "3.5em"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>"], value='not set', rows=1)#, style={"description width": "8em"})
                
        self.set_widget(w.HBox([self.__condition, self.__element]))
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [5]:
class Float_Criterion (Criterion):
    """ This class allows for representing criterions based on float value """
 
    def __init__ (self, key: str, description: str, min_value: float, max_value: float):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.BoundedFloatText(value=min_value, min=min_value, max=max_value, layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>","<=","<",">",">="], value='not set', rows=1, layout={"width": "auto"})
                
        self.set_widget(w.HBox([self.__condition, self.__element]))
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [6]:
class String_Criterion (Criterion):
    """ This class allows for representing criterions based on string value """
 
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.Text(layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>"], value='not set', rows=1)
                
        self.set_widget(w.HBox([self.__condition, self.__element]))
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [7]:
class Item_Criterion (Criterion):
    """ This class allows for representing criterions based on list of values """
 
    def __init__ (self, key: str, description: str, values: list):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.Select(options=values, description=description, disabled=False,rows=1)
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>"], value='not set', rows=1)
                
        self.set_widget(w.HBox([self.__condition, self.__element]))
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [8]:
class Query_Criterion (Criterion):
    """ This class allows for representing the desired query """
 
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.Select(description=self.get_description(), options=["benzenoids","ir","ims2d1a","nics","clar_covers","properties","irregularities"], value='benzenoids', rows=1, layout={"width": "auto"})
                
        self.set_widget(w.HBox([self.__element]))
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        return str(self.__element.value)

In [9]:
class View_Criterion (Criterion):
    """ This class allows for representing the desired view """
 
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.Select(description=self.get_description(), options=["global","one by one"], value='global', rows=1, layout={"width": "auto"})
                
        self.set_widget(w.HBox([self.__element]))
           
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        return str(self.__element.value)

### Definition of the display classes

In [10]:
class Display:
    """ This class allows for displaying benzenoid information """
    
    def __init__ (self, info: dict):
        """ initializes the display tool """
        self.__information = info
        self.__data = {}
        self.add_data ("Benzenoid id", self.get_information("idBenzenoid"))
        self.add_data ("InChI", self.get_information("inchi"))
        self.add_data ("Label", self.get_information("label"))
        self.add_data ("#hexagons", self.get_information("nbHexagons"))
        self.add_data ("#carbons", self.get_information("nbCarbons"))
        self.add_data ("#hydrogens", self.get_information("nbHydrogens"))
        self.add_data ("Weight", self.get_information("weight"))
        self.add_data ("Irregularity", self.get_information("irregularity"))
    
    
    def add_data (self, label, value) -> None:
        """ adds a label to display with its corresponding value """
        self.__data [label] = [value]
        
    
    def display (self) -> None:
        """ displays the information """           
        self.__df = pd.DataFrame(self.__data)
        self.__df.index = [""]
        
        ip.display.display(self.__df.transpose())
        self.display_molecule()
        
    
    def display_molecule (self) -> None:
        """ displays the molecule thanks to Jsmol """
        view = Jsmol.JsmolView.from_str(str(self.get_information("nbCarbons")+self.get_information("nbHydrogens"))+"\nComment\n"+self.get_information("geometry"))
        ip.display.display(view)
        
    
    def display_image (self, str64: str) -> None:
        """ displays the base-64 image defined by str64 """
        img = Image.open(io.BytesIO(base64.b64decode(str64)))
        ip.display.display(img)
        
    
    def get_information (self, key) -> dict:
        """ returns the information """
        return self.__information[key]

In [11]:
class Display_IR (Display):
    """ This class allows for displaying benzenoid information from IR query """

    def __init__ (self, info: dict):
        """ initializes the display tool """
        super().__init__(info)
        self.add_data ("Final energy", self.get_information("finalEnergy"))
        
    
    def display (self) -> None:
        """ displays the information """
        super().display()
               
        x = [float(v) for v in self.get_information("frequencies").split(" ")]
        y = [float(v) for v in self.get_information("intensities").split(" ")]
        
        plt.cla()
        plt.stem(x, y, linefmt='-', markerfmt="")
        plt.xlabel('frequencies ($cm^{-1}$)')
        plt.ylabel('intensities ($km.mol^{-1}$)')
        plt.show()
        download_button("AMES", "ames.xml", self.get_information("amesFormat"))

In [12]:
class Display_IMS2D1A (Display):
    """ This class allows for displaying benzenoid information from IMS2D1A query """

    def __init__ (self, info: dict):
        """ initializes the display tool """
        super().__init__(info)
        self.add_data ("Type", self.get_information("type"))

    def display (self) -> None:
        """ displays the information """
        super().display()
        self.display_image(self.get_information("picture"))

In [13]:
class Display_NICS (Display):
    """ This class allows for displaying benzenoid information from NICS query """
    def __init__ (self, info: dict):
        """ initializes the display tool """
        super().__init__(info)
        self.add_data ("NICS values", self.get_information("nics"))

In [14]:
class Display_Clar_Covers (Display):
    """ This class allows for displaying benzenoid information from Clar Cover query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        self.display_image(self.get_information("clarCover"))

In [15]:
class Display_Properties (Display):
    """ This class allows for displaying benzenoid information from properties query """

    def __init__ (self, info: dict):
        """ initializes the display tool """
        super().__init__(info)
        yesno = ["no","yes"]
        self.add_data ("Catacondensed", yesno[self.get_information("catacondensed")])
        self.add_data ("Coronoid", yesno[self.get_information("coronoid")])
        self.add_data ("Coronenoid", yesno[self.get_information("coronenoid")])
        self.add_data ("Symmetry", self.get_information("symmetry"))
        self.add_data ("#Kekulé structures", self.get_information("kekule"))
        self.add_data ("HOMO", self.get_information("homo"))
        self.add_data ("LUMO", self.get_information("lumo"))
        self.add_data ("Dipole moment", self.get_information("moment"))

In [16]:
class Display_Irregularities (Display):
    """ This class allows for displaying benzenoid information from irregularities query """
    
    def __init__ (self, info: dict):
        """ initializes the display tool """
        super().__init__(info)
        self.add_data ("# solo", self.get_information("solo"))
        self.add_data ("# duo", self.get_information("duo"))
        self.add_data ("# trio", self.get_information("trio"))
        self.add_data ("# quartet", self.get_information("quartet"))        

### Definition of the table classes

In [17]:
class Table:
    """ This class allows for displaying a table containing benzenoid information """
    
    def __init__ (self):
        """ initializes the display tool """
        self.__data = {}
    
    
    def add_data (self, label, value) -> None:
        """ adds a label to display with its corresponding value """
        if label not in self.__data:
            self.__data[label] = []
        self.__data [label] += [value]
        
    
    def add_row (self, info: dict) -> None:
        """ adds a row based on the given information """           
        self.add_data ("Benzenoid id", info["idBenzenoid"])
        self.add_data ("InChI", info["inchi"])
        self.add_data ("Label", info["label"])
        self.add_data ("#hexagons", info["nbHexagons"])
        self.add_data ("#carbons", info["nbCarbons"])
        self.add_data ("#hydrogens", info["nbHydrogens"])
        self.add_data ("Weight", info["weight"])
        self.add_data ("Irregularity", info["irregularity"])
        
        
    def print (self):
        """ prints the table """
        self.__df = pd.DataFrame(self.__data)
        ip.display.display(self.__df)

In [18]:
class Table_Properties (Table):
    """ This class allows for displaying a table containing benzenoid information from properties query """

    def add_row (self, info: dict) -> None:
        """ adds a row based on the given information """
        super().add_row (info)
        yesno = ["no","yes"]
        self.add_data ("Catacondensed", yesno[info["catacondensed"]])
        self.add_data ("Coronoid", yesno[info["coronoid"]])
        self.add_data ("Coronenoid", yesno[info["coronenoid"]])
        self.add_data ("Symmetry", info["symmetry"])
        self.add_data ("#Kekulé structures", info["kekule"])
        self.add_data ("HOMO", info["homo"])
        self.add_data ("LUMO", info["lumo"])
        self.add_data ("Gap", info["homo"]-info["lumo"])
        self.add_data ("Dipole moment", info["moment"])
        

In [19]:
class Table_Irregularities (Table):
    """ This class allows for displaying a table containing benzenoid information from irregularities query """
    
    def add_row (self, info: dict) -> None:
        """ adds a row based on the given information """
        super().add_row (info)
        self.add_data ("# solo", info["solo"])
        self.add_data ("# duo", info["duo"])
        self.add_data ("# trio", info["trio"])
        self.add_data ("# quartet", info["quartet"])
        

### Definition of the form class

In [20]:
def download_button (label: str, filename: str, data: str) -> None:
    """ prints a download button """
    # we encode the data
    encoded_data = base64.b64encode(data.encode()).decode()
    
    # we create the download button
    button = '''<html>
    <body>
    <a download="'''+filename+'''" href="data:text/'''+filename.split(".")[1]+''';base64,'''+encoded_data+'''" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download '''+label+''' file</button>
    </a>
    </body>
    </html>
    '''
    
    ip.display.display(w.HTML(button))

    
def process (form):    
    demand_type = form.get_demand_type()

    if demand_type == "data":
        data = form.get_data()
        query = form.get_query()
        view = form.get_view()
        
        if view == "global":
            if query == "properties":
                d = Table_Properties()
                data = sorted(data, key=lambda t: (t["nbHexagons"],t["symmetry"],t["homo"]-t["lumo"]))
            elif query == "irregularities":
                d = Table_Irregularities()
            else:
                d = Table()
            for molecule in data:
                d.add_row (molecule)
            d.print()
        elif view == "one by one":
            for molecule in data:
                if query == "benzenoids":
                    d = Display(molecule)
                elif query == "ir":
                    d = Display_IR(molecule)
                elif query == "ims2d1a":
                    d = Display_IMS2D1A(molecule)
                elif query == "nics":
                    d = Display_NICS(molecule)
                elif query == "clar_covers":
                    d = Display_Clar_Covers(molecule)
                elif query == "properties":
                    d = Display_Properties(molecule)
                elif query == "irregularities":
                    d = Display_Irregularities(molecule)
                d.display()

    elif demand_type == "count":
        print ("Number of molecules:",form.get_data())

    elif demand_type == "json":
        download_button("JSON", "query.json", form.get_json_string())
        print ("JSON query:")
        print (form.get_json_string())
        
    elif demand_type == "result":
        download_button("JSON", "result.json", str(form.get_data()))
        print ("JSON result (100,000 first characters):")
        print (str(form.get_data())[0:100_000])

In [21]:
class Form:
    
    def __init__ (self):
        """ initialise the form """
        self.__query = None         # the query
        self.__demand_type = None   # the type of demand
        self.__json_string = None   # the query as a JSON string
        self.__data = None          # the data related to the query (if the query succeeds)
        
        self.create_form()
        
        
    def create_form(self) -> None:
        """ creates the form allowing for choosing the values of desired criteria """
        self.__criteria = []

        # criteria about basic information
        self.__criteria.append (Int_Criterion(key="idBenzenoid", description="BenzDB id", min_value=0, max_value=8391))
        self.__criteria.append (String_Criterion(key="inchi", description="InChI"))
        self.__criteria.append (String_Criterion(key="label", description="BenzAI label"))
        
        self.__criteria.append (Int_Criterion(key="nbHexagons", description="# hexagons", min_value=1, max_value=9))
        self.__criteria.append (Int_Criterion(key="nbCarbons", description="# carbons", min_value=6, max_value=38))
        self.__criteria.append (Int_Criterion(key="nbHydrogens", description="# hydrogens", min_value=6, max_value=22))
                
        self.__criteria.append (Int_Criterion(key="solo", description="# solo", min_value=0, max_value=14))
        self.__criteria.append (Int_Criterion(key="duo", description="# duo", min_value=0, max_value=14))
        self.__criteria.append (Int_Criterion(key="trio", description="# trio", min_value=0, max_value=18))
        self.__criteria.append (Int_Criterion(key="quartet", description="# quartet", min_value=0, max_value=20))
        self.__criteria.append (Float_Criterion(key="irregularity", description="Irregularity", min_value=0, max_value=1))
        
        self.__criteria.append (Int_Criterion(key="kekule", description="# structures", min_value=0, max_value=110))
        
        self.__criteria.append (Boolean_Criterion(key="catacondensed", description="Is catacondensed"))
        self.__criteria.append (Boolean_Criterion(key="coronenoid", description="Is coronenoid"))
        self.__criteria.append (Boolean_Criterion(key="coronoid", description="Is coronoid"))
        
        self.__criteria.append (Float_Criterion(key="frequency", description="Frequency", min_value=0, max_value=3466))
        self.__criteria.append (Float_Criterion(key="intensity", description="Intensity", min_value=0, max_value=2742))
        
        self.__criteria.append (Item_Criterion(key="symmetry", description="Symmetry", values=["Cs","C3hi","C3hii","C2va","C2vb","C2hi","C2hii","D2hi","D2hii","D3hia","D3hii","D6h"]))

        # definition of groups of widgets
        self.add_group ([self.__criteria[0:1],self.__criteria[1:3]], "Molecule identity")
        self.add_group ([self.__criteria[3:6]], "Basic features")
        self.add_group ([self.__criteria[6:10],self.__criteria[10:11]],"Irregularity features")
        self.add_group ([self.__criteria[11:12]],"Kekulé structures")
        self.add_group ([self.__criteria[12:15]],"Structural properties")
        self.add_group ([self.__criteria[15:17]],"IR features")
        self.add_group ([self.__criteria[17:18]],"Symmetry property")
        
        # query
        self.__criteria.append (Query_Criterion(key="query", description="query"))
        
        # type of view
        self.__criteria.append (View_Criterion(key="view", description="View type"))

        # validation buttons
        count_btn = w.Button (description="Count")
        count_btn.on_click (self.perform_query)

        getdata_btn = w.Button (description="Get Data")
        getdata_btn.on_click (self.perform_query)

        getquery_btn = w.Button (description="Get JSON Query")
        getquery_btn.on_click (self.perform_query)
        
        getresult_btn = w.Button (description="Get JSON Result")
        getresult_btn.on_click (self.perform_query)
        
        self.add_group([self.__criteria[18:20],[count_btn, getdata_btn, getquery_btn, getresult_btn]],"Query choices")
    
    
    def perform_query (self, btn) -> None:
        """ performs the query and sets the corresponding attributes """ 
        # we identify the type of demand
        if btn.description == "Count":
            self.__demand_type = "count"
        elif btn.description == "Get Data":
            self.__demand_type = "data"
        elif btn.description == "Get JSON Query":
            self.__demand_type = "json"
        elif btn.description == "Get JSON Result":
            self.__demand_type = "result"
        else:
            self.__demand_type = "unknown"

        # we build the JSON string
        self.__json_string = "{\n"
        for c in self.__criteria:
            if isinstance(c,Query_Criterion):
                self.__query = c.get_criterion()
            elif isinstance(c,View_Criterion):
                self.__view = c.get_criterion()
            else:
                s = c.get_criterion()
                if len(s) > 0:
                    if len(self.__json_string) > 3:
                        self.__json_string += ",\n"
                    self.__json_string += "\t" + s
        self.__json_string += "\n}"

        if self.__demand_type in ["data", "result"]:
            response = requests.post("https://benzenoids.lis-lab.fr/find_"+self.__query, json= json.loads(self.__json_string))    
        elif self.__demand_type == "count":
            response = requests.post("https://benzenoids.lis-lab.fr/count_"+self.__query, json= json.loads(self.__json_string))
        
        if self.__demand_type in ["data", "count", "result"]:
            if response.status_code == 200:
                self.__data = response.json()
                
        process(self)

        
    def add_group (self, criteria:list, title: str) -> None:
        """ defines a group of widgets from the given list of criteria """
        title_box = w.HTML(value="<b>"+title+"</b>")
        display(title_box)

        for l in criteria:
            box = w.HBox([c.get_widget() if isinstance(c,Criterion) else c for c in l])
            display(box)
        
        
    def get_data (self):
        """ returns the data related to the query """
        return self.__data
    
    
    def get_query (self):
        """ returns the desired query """
        return self.__query

    
    def get_json_string (self):
        """ returns the JSON string related to the desired query """
        return self.__json_string
    

    def get_demand_type (self):
        """ returns the type of the current demand """
        return self.__demand_type
    
    
    def get_view (self):
        """ returns the desired type of view """
        return self.__view

## Filling the form

In [23]:
form = Form()


HTML(value='<b>Molecule identity</b>')

HTML(value='<b>Basic features</b>')

HTML(value='<b>Irregularity features</b>')

HTML(value='<b>Kekulé structures</b>')

HTML(value='<b>Structural properties</b>')

HTML(value='<b>IR features</b>')

HTML(value='<b>Symmetry property</b>')

HTML(value='<b>Query choices</b>')

,Benzenoid id,InChI,Label,#hexagons,#carbons,#hydrogens,Weight,Irregularity,Catacondensed,Coronoid,Coronenoid,Symmetry,#Kekulé structures,HOMO,LUMO,Gap,Dipole moment
0,1536,1S/C6H6/c1-2-4-6-5-3-1/h1-6H,0,1,6,6,78.11208,-1.0000,yes,no,yes,D6h,2,-0.24820,0.00739,-0.25559,0.00000
1,8332,1S/C10H8/c1-2-6-10-8-4-3-7-9(10)5-1/h1-8H,0-4,2,10,8,128.17092,1.0000,yes,no,no,D2hii,3,-0.21411,-0.03263,-0.18148,0.00000
2,3532,1S/C14H10/c1-3-7-13-11(5-1)9-10-12-6-2-4-8-14(...,1-3-4,3,14,10,178.22976,0.8000,yes,no,no,C2va,5,-0.21206,-0.03350,-0.17856,1.71439
3,4421,1S/C14H10/c1-2-6-12-10-14-8-4-3-7-13(14)9-11(1...,0-4-8,3,14,10,178.22976,0.8000,yes,no,no,D2hi,4,-0.19312,-0.05816,-0.13496,0.00000
4,4674,1S/C13H22/c1-4-10-6-2-8-12-9-3-7-11(5-1)13(10)...,0-3-4,3,13,9,165.21108,1.0000,no,no,no,D3hii,0,-0.16193,-0.00267,-0.15926,0.00000
5,5239,1S/C18H12/c1-3-7-15-13(5-1)9-11-18-16-8-4-2-6-...,0-6-11-17,4,18,12,228.28860,0.6667,yes,no,no,C2hii,8,-0.20390,-0.04391,-0.15999,0.00000
6,6092,1S/C18H12/c1-3-7-16-13(5-1)9-11-15-12-10-14-6-...,0-6-10-11,4,18,12,228.28860,0.6667,yes,no,no,C2vb,8,-0.20508,-0.04451,-0.16057,0.63130
7,5036,1S/C18H12/c1-2-7-15-12-18-16(11-14(15)6-1)10-9...,0-6-12-17,4,18,12,228.28860,0.6667,yes,no,no,Cs,7,-0.19678,-0.05473,-0.14205,0.83463
8,5436,1S/C17H28/c1-2-9-15-13(5-1)11-14-8-3-6-12-7-4-...,1-5-6-11,4,17,11,215.26992,0.9091,no,no,no,Cs,0,-0.16030,-0.03288,-0.12742,0.53087
9,8010,1S/C16H10/c1-3-11-7-9-13-5-2-6-14-10-8-12(4-1)...,0-5-6-11,4,16,10,202.25124,0.6000,no,no,no,D2hii,6,-0.19677,-0.05245,-0.14432,0.00000
